# Create satisfactory db

Using json from Tomáš Blatný [Satisfacroy Tools](https://github.com/greeny/SatisfactoryTools/tree/dev/data)
Dowload data.json,
Rename it satisfactory.json

In [1]:
import json
import os

from sqlalchemy import create_engine, Table, Column, Integer, String, Boolean, MetaData, Float

In [2]:
with open('satisfactory.json') as datas:
    datas = json.load(datas)
    
for i in datas:
    print(i)
    
recipes_src = datas['recipes']
items_src = datas['items']
schematics_src = datas['schematics']
generators_src = datas['generators']
resources_src = datas['resources']
miners_src = datas['miners']
buildings_src = datas['buildings']

for i in items_src.items():
    print(i)
    break
    
for i in recipes_src.items():
    print(i)
    break
    
for i in recipes_src.values():
    if i['forBuilding']:
        print(i)
        break
    
for i in buildings_src.items():
    print(i)
    break

recipes
items
schematics
generators
resources
miners
buildings
('BP_EqDescZipLine_C', {'slug': 'zipline', 'className': 'BP_EqDescZipLine_C', 'name': 'Zipline', 'sinkPoints': 5284, 'description': 'Slot: Hand\n\nProvides faster traversal in factories by allowing Pioneers to zip along Power lines.\nActivate the Zipline and aim at a nearby Power Line to connect to it.', 'stackSize': 1, 'energyValue': 0, 'radioactiveDecay': 0, 'liquid': False, 'fluidColor': {'r': 0, 'g': 0, 'b': 0, 'a': 0}})
('Recipe_AILimiter_C', {'slug': 'ai-limiter', 'name': 'AI Limiter', 'className': 'Recipe_AILimiter_C', 'alternate': False, 'time': 12, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_CopperSheet_C', 'amount': 5}, {'item': 'Desc_HighSpeedWire_C', 'amount': 20}], 'forBuilding': False, 'inMachine': True, 'inHand': True, 'inWorkshop': False, 'products': [{'item': 'Desc_CircuitBoardHighSpeed_C', 'amount': 1}], 'producedIn': ['Desc_AssemblerMk1_C']})
{'slug': 'assembler', 'name': 'Assembler', 'class

In [3]:
satisfactory_db = "satisfactory.db"

if os.path.exists(satisfactory_db):
    os.unlink(satisfactory_db)

In [4]:
engine = create_engine(f'sqlite:///{satisfactory_db}')

In [5]:
meta = MetaData()

recipes = Table(
    'recipes', meta, 
    Column('id', Integer, primary_key = True), 
    Column('slug', String), 
    Column('name', String), 
    Column('className', String), 
    Column('alternate', Boolean), 
    Column('time', Float), 
    Column('manualTimeMultiplier', Integer), 
    Column('forBuilding', Boolean), 
    Column('inMachine', Boolean), 
    Column('inHand', Boolean), 
    Column('inWorkshop', Boolean),
    Column('producedIn', String)
)

items = Table(
    'items', meta,
    Column('id', Integer, primary_key = True),
    Column('slug', String), 
    Column('name', String), 
    Column('className', String), 
    Column('sinkPoints', Integer),
    Column('description', String),
    Column('stackSize', Integer),
    Column('energyValue', Integer),
    Column('radioactiveDecay', Integer),
    Column('liquid', Boolean)
)

recipe_ingredients = Table(
    'recipe_ingredients', meta,
    Column('id', Integer, primary_key = True),
    Column('recipe', Integer),
    Column('item', Integer),
    Column('amount', Integer)
)

recipe_products = Table(
    'recipe_products', meta,
    Column('id', Integer, primary_key = True),
    Column('recipe', Integer),
    Column('item', Integer),
    Column('amount', Integer)
)

buildings = Table(
    'buildings', meta,
    Column('id', Integer, primary_key = True),
    Column('slug', String), 
    Column('name', String), 
    Column('description', String), 
    # Not categories
    Column('buildMenuPriority', Integer),
    Column('className', String),
    # Not metadata
    # Not size but: (0 for now in source !) 
    Column('width', Integer),
    Column('length', Integer),
    Column('height', Integer),
    Column('recipe', Integer)
)

meta.create_all(engine)

In [6]:
for i in items_src.values():
    if 'fluidColor' in i: del i['fluidColor']
    ins = items.insert().values(i)

    engine.execute(ins)

In [7]:
for b in buildings_src.values():
    if 'categories' in b:
        del b['categories']
    if 'metadata' in b:
        del b['metadata']
    if 'size' in b:
        for desc in b['size']:
            b[desc] = b['size'][desc]
        del b['size']
    
    ins = buildings.insert().values(b)
    
    engine.execute(ins)

In [8]:
for r in recipes_src.values():
    
    c = dict(r)
    del c['ingredients']
    del c['products']
    if len(c['producedIn']) > 0:
        if len(c['producedIn']) > 1:
            print(c['slug'])
        c['producedIn'] = c['producedIn'][0]
    else:
        c['producedIn'] = None
    
    ins = recipes.insert().values(c)
    a = engine.execute(ins)
    key = a.inserted_primary_key[0]
    for ing in r['ingredients']:
        it = engine.execute(items.select().where(items.c.className == ing['item'])).fetchone()
        if it is not None:
            ing_id = it[0]
            ins = recipe_ingredients.insert().values(recipe = key, item = ing_id, amount = ing['amount'])
            engine.execute(ins)
        
    if r['forBuilding']:
        for prod in r['products']:
            assert prod['amount'] == 1
            ins = (buildings.update()
                   .where(buildings.c.className == prod['item'])
                   .values(recipe = key))
            engine.execute(ins)
    else:
        for prod in r['products']:
            it = engine.execute(items.select().where(items.c.className == prod['item'])).fetchone()
            if it is not None:
                prod_id = it[0]
                ins = recipe_products.insert().values(recipe = key, item = prod_id, amount = prod['amount'])
                engine.execute(ins)
            
    if r['forBuilding']:
        print(f'{key}: {r}')


94: {'slug': 'assembler', 'name': 'Assembler', 'className': 'Recipe_AssemblerMk1_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_IronPlateReinforced_C', 'amount': 8}, {'item': 'Desc_Rotor_C', 'amount': 4}, {'item': 'Desc_Cable_C', 'amount': 10}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_AssemblerMk1_C', 'amount': 1}], 'producedIn': []}
104: {'slug': 'blender', 'name': 'Blender', 'className': 'Recipe_Blender_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_Motor_C', 'amount': 20}, {'item': 'Desc_ModularFrameHeavy_C', 'amount': 10}, {'item': 'Desc_AluminumCasing_C', 'amount': 50}, {'item': 'Desc_ModularFrameLightweight_C', 'amount': 5}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_Blender_C', 'amount': 1}], 'producedIn': []}
107: {'slug': 'ceiling-light', 'name': 'Ceiling L

137: {'slug': 'drone-port', 'name': 'Drone Port', 'className': 'Recipe_DroneStation_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_ModularFrameHeavy_C', 'amount': 20}, {'item': 'Desc_HighSpeedConnector_C', 'amount': 20}, {'item': 'Desc_AluminumPlate_C', 'amount': 50}, {'item': 'Desc_AluminumCasing_C', 'amount': 50}, {'item': 'Desc_ModularFrameLightweight_C', 'amount': 10}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_DroneStation_C', 'amount': 1}], 'producedIn': []}
138: {'slug': 'drone', 'name': 'Drone', 'className': 'Recipe_DroneTransport_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_Motor_C', 'amount': 4}, {'item': 'Desc_AluminumPlate_C', 'amount': 10}, {'item': 'Desc_ModularFrameLightweight_C', 'amount': 1}, {'item': 'Desc_CircuitBoardHighSpeed_C', 'amount': 2}, {'item': 'BP_ItemDescriptorPortableMiner_C', 'amount': 1}], 'forBui

171: {'slug': 'stackable-hyper-tube-support', 'name': 'Stackable Hyper Tube Support', 'className': 'Recipe_HyperPoleStackable_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_IronPlate_C', 'amount': 4}, {'item': 'Desc_IronRod_C', 'amount': 2}, {'item': 'Desc_Cement_C', 'amount': 2}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_HyperPoleStackable_C', 'amount': 1}], 'producedIn': []}
172: {'slug': 'hyper-tube-wall-hole', 'name': 'Hyper Tube Wall Hole', 'className': 'Recipe_HyperTubeWallHole_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_IronPlate_C', 'amount': 2}, {'item': 'Desc_Cement_C', 'amount': 2}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_HyperTubeWallHole_C', 'amount': 1}], 'producedIn': []}
173: {'slug': 'hyper-tube-wall-support', 'name': 'Hyper Tube Wall Support',

228: {'slug': 'fluid-buffer', 'name': 'Fluid Buffer', 'className': 'Recipe_PipeStorageTank_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_CopperSheet_C', 'amount': 10}, {'item': 'Desc_ModularFrame_C', 'amount': 5}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_PipeStorageTank_C', 'amount': 1}], 'producedIn': []}
229: {'slug': 'stackable-pipeline-support', 'name': 'Stackable Pipeline Support', 'className': 'Recipe_PipeSupportStackable_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_IronPlate_C', 'amount': 4}, {'item': 'Desc_IronRod_C', 'amount': 2}, {'item': 'Desc_Cement_C', 'amount': 2}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_PipeSupportStackable_C', 'amount': 1}], 'producedIn': []}
230: {'slug': 'pipeline-wall-hole', 'name': 'Pipeline Wall Hole', 'className': 'Recipe

263: {'slug': 'inverted-outer-corner-quarter-pipe', 'name': 'Inverted Outer Corner Quarter Pipe', 'className': 'Recipe_QuarterPipeCorner_04_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_Cement_C', 'amount': 6}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_QuarterPipeCorner_04_C', 'amount': 1}], 'producedIn': []}
264: {'slug': 'inverted-quarter-pipe', 'name': 'Inverted Quarter Pipe', 'className': 'Recipe_QuarterPipe_02_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_Cement_C', 'amount': 6}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_QuarterPipe_02_C', 'amount': 1}], 'producedIn': []}
265: {'slug': 'quarter-pipe', 'name': 'Quarter Pipe', 'className': 'Recipe_QuarterPipe_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_Cement_C',

296: {'slug': 'awesome-shop', 'name': 'AWESOME Shop', 'className': 'Recipe_ResourceSinkShop_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_IronScrew_C', 'amount': 200}, {'item': 'Desc_IronPlate_C', 'amount': 10}, {'item': 'Desc_Cable_C', 'amount': 10}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_ResourceSinkShop_C', 'amount': 1}], 'producedIn': []}
297: {'slug': 'awesome-sink', 'name': 'AWESOME Sink', 'className': 'Recipe_ResourceSink_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_IronPlateReinforced_C', 'amount': 15}, {'item': 'Desc_Cable_C', 'amount': 30}, {'item': 'Desc_Cement_C', 'amount': 45}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_ResourceSink_C', 'amount': 1}], 'producedIn': []}
302: {'slug': 'smelter', 'name': 'Smelter', 'className': 'Recipe_SmelterBasicMk1

345: {'slug': 'valve', 'name': 'Valve', 'className': 'Recipe_Valve_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_Rubber_C', 'amount': 4}, {'item': 'Desc_SteelPlate_C', 'amount': 4}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_Valve_C', 'amount': 1}], 'producedIn': []}
346: {'slug': 'walkway-crossing', 'name': 'Walkway Crossing', 'className': 'Recipe_Walkway_Cross_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_IronRod_C', 'amount': 2}, {'item': 'Desc_IronPlate_C', 'amount': 1}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_WalkwayCross_C', 'amount': 1}], 'producedIn': []}
347: {'slug': 'walkway-ramp', 'name': 'Walkway Ramp', 'className': 'Recipe_Walkway_Ramp_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_IronRod_C', 'amount': 

371: {'slug': 'panel-window', 'name': 'Panel Window', 'className': 'Recipe_Wall_Window_8x4_03_Steel_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_IronPlate_C', 'amount': 3}, {'item': 'Desc_Cement_C', 'amount': 3}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_Wall_Window_8x4_03_Steel_C', 'amount': 1}], 'producedIn': []}
372: {'slug': 'reinforced-window', 'name': 'Reinforced Window', 'className': 'Recipe_Wall_Window_8x4_04_C', 'alternate': False, 'time': 1, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_IronPlate_C', 'amount': 3}, {'item': 'Desc_Cement_C', 'amount': 3}], 'forBuilding': True, 'inMachine': False, 'inHand': False, 'inWorkshop': False, 'products': [{'item': 'Desc_Wall_Window_8x4_04_C', 'amount': 1}], 'producedIn': []}
373: {'slug': 'water-extractor', 'name': 'Water Extractor', 'className': 'Recipe_WaterPump_C', 'alternate': False, 'time': 1, 'manualTimeMul

In [38]:
recipes_src["Recipe_IronRod_C"]

{'slug': 'iron-rod',
 'name': 'Iron Rod',
 'className': 'Recipe_IronRod_C',
 'alternate': False,
 'time': 4,
 'manualTimeMultiplier': 0.5,
 'ingredients': [{'item': 'Desc_IronIngot_C', 'amount': 1}],
 'forBuilding': False,
 'inMachine': True,
 'inHand': True,
 'inWorkshop': False,
 'products': [{'item': 'Desc_IronRod_C', 'amount': 1}],
 'producedIn': ['Desc_ConstructorMk1_C']}

In [39]:
def make_part(name, stackSize):
    return {
        'slug': name.lower().replace(' ','-'),
        'className': 'Desc_' + name.replace(' ','') + '_C',
        'name': name,
        'sinkPoints': 0,
        'description': 'no description',
        'stackSize': stackSize,
        'energyValue': 0,
        'radioactiveDecay': 0,
        'liquid': False,
        'fluidColor': {'r': 0, 'g': 0, 'b': 0, 'a': 0}}

def make_recipe(name, time):
    return {
        'slug': name.lower().replace(' ','-'),
        'name': name,
        'className': 'Desc_' + name.replace(' ','') + '_C',
        'alternate': False,
        'time': time,
        'manualTimeMultiplier': 1 ,
        'ingredients': [{'item': 'Desc_IronIngot_C', 'amount': 1}],
        'forBuilding': False,
        'inMachine': True,
        'inHand': True,
        'inWorkshop': False,
        'products': [{'item': 'Desc_IronRod_C', 'amount': 1}],
        'producedIn': ['Desc_ConstructorMk1_C']}

In [40]:
xmas_parts = [ make_part(name, stack) for name, stack in
    [
        ('FICSMAS Gift', 500), ('Candy Cane', 500), ('Actual Snow', 500),
        ('Red FICSMAS Ornament', 500), ('Blue FICSMAS Ornament', 500),
        ('Copper FICSMAS Ornament', 200), ('Iron FICSMAS Ornament', 200),
        ('FICSMAS Ornament Bundle', 100),
        ('FICSMAS Bow', 500),
        ('FICSMAS Tree Branch', 500),
        ('FICSMAS Wonder Star', 50),
        ('FICSMAS Decoration', 100),
        ('Snowball', 500)
    ]]

In [41]:
xmas_recipes = [
    {'slug': 'iron-rod',
 'name': 'Iron Rod',
 'className': 'Recipe_IronRod_C',
 'alternate': False,
 'time': 4,
 'manualTimeMultiplier': 0.5,
 'ingredients': [{'item': 'Desc_IronIngot_C', 'amount': 1}],
 'forBuilding': False,
 'inMachine': True,
 'inHand': True,
 'inWorkshop': False,
 'products': [{'item': 'Desc_IronRod_C', 'amount': 1}],
 'producedIn': ['Desc_ConstructorMk1_C']}
]

Correction